# non-combinatorial example

In [1]:
using DynamicPolynomials
using HomotopyContinuation
using Arblib

In [2]:
@polyvar w[1:2] x[1:2] y[1:2] a[1:2] b[1:2] λᴿ λᴵ s t ν ν₁ ν₂

(PolyVar{true}[w₁, w₂], PolyVar{true}[x₁, x₂], PolyVar{true}[y₁, y₂], PolyVar{true}[a₁, a₂], PolyVar{true}[b₁, b₂], λᴿ, λᴵ, s, t, ν, ν₁, ν₂)

In [3]:
H = (1-w[1]-w[2])*(20-w[1]-40*w[2])-1
sys = System([H, w[1]*differentiate(H, w[1]) - w[2]*differentiate(H, w[2])])
result = solve(sys)
display(map(sol -> round.(sol; digits=3), solutions(result)))

Tracking 4 paths... 100%|███████████████████████████████| Time: 0:00:06
  # paths tracked:                  4
  # non-singular solutions (real):  4 (2)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         4 (2)


4-element Vector{Vector{ComplexF64}}:
 [0.49 - 0.281im, 0.581 + 0.162im]
 [0.49 + 0.281im, 0.581 - 0.162im]
 [0.548 + 0.0im, 0.31 + 0.0im]
 [9.997 + 0.0im, 0.253 + 0.0im]

In [4]:
# for convenience we first write H = u(a, b) + iv(a, b)
@polyvar I
u = sum(filter(t -> iseven(degree(t, I)), terms(H(w => a + I.*b))))
v = sum(filter(t -> isodd(degree(t, I)), terms(H(w => a + I.*b))))
u, v = subs(u, I => im), -im*subs(v, I => im)

(a₁² + (41 + 0im)a₁a₂ + (40 + 0im)a₂² + -b₁² + (-41 + 0im)b₁b₂ + (-40 + 0im)b₂² + (-21 + 0im)a₁ + (-60 + 0im)a₂ + (19 + 0im), (2 + 0im)a₁b₁ + (41 + 0im)a₁b₂ + (41 + 0im)a₂b₁ + (80 + 0im)a₂b₂ + (-21 + 0im)b₁ + (-60 + 0im)b₂)

In [5]:
# critical equations in variables a, b, λᴿ λᴵ after splitting
criteqs = [ u; v; a.*differentiate(u, a) + b.*differentiate(u, b) .- λᴿ;
                  a.*differentiate(v, a) + b.*differentiate(v, b) .- λᴵ ]
# minimality equations in variables x, y, a, b, t
circeqs = [u([a; b] => [x; y]); v([a; b] => [x; y]); x.^2 + y.^2 - t.*(a.^2 + b.^2)]
# balance equations
J2eqs = (ν₁.*y - ν₂.*x) .* differentiate(u([a; b] => [x; y]), x) - 
        (ν₁.*x + ν₂.*y) .* differentiate(u([a; b] => [x; y]), y)
# # there are two systems to solve depending on the value of ν₁
extended_systems = [
    System([criteqs; circeqs; s*(1-t) - 1; subs(J2eqs, [ν₁, ν₂] => [1, ν])]),
    System([criteqs; circeqs; s*(1-t) - 1; subs(J2eqs[1], [ν₁, ν₂] => [0, 1])])
]

2-element Vector{System}:
 System of length 13
 13 variables: x₁, x₂, y₁, y₂, a₁, a₂, b₁, b₂, λᴿ, λᴵ, s, t, ν

 19 - 21*a₁ - 60*a₂ + 41*a₁*a₂ - 41*b₂*b₁ + a₁^2 + 40*a₂^2 - b₁^2 - 40*b₂^2
 -21*b₁ - 60*b₂ + 2*b₁*a₁ + 41*b₁*a₂ + 41*b₂*a₁ + 80*b₂*a₂
 -21*a₁ - λᴿ + 41*a₁*a₂ - 41*b₂*b₁ + 2*a₁^2 - 2*b₁^2
 -60*a₂ - λᴿ + 41*a₁*a₂ - 41*b₂*b₁ + 80*a₂^2 - 80*b₂^2
 -21*b₁ - λᴵ + 4*b₁*a₁ + 41*b₁*a₂ + 41*b₂*a₁
 -60*b₂ - λᴵ + 41*b₁*a₂ + 41*b₂*a₁ + 160*b₂*a₂
 19 - 21*x₁ - 60*x₂ + 41*x₂*x₁ - 41*y₁*y₂ + x₁^2 + 40*x₂^2 - y₁^2 - 40*y₂^2
 -21*y₁ - 60*y₂ + 2*y₁*x₁ + 41*y₁*x₂ + 41*y₂*x₁ + 80*y₂*x₂
 -t*a₁^2 - t*b₁^2 + x₁^2 + y₁^2
 -t*a₂^2 - t*b₂^2 + x₂^2 + y₂^2
 -1 + s - s*t
 -21*y₁ + 4*y₁*x₁ + 41*y₁*x₂ + 41*y₂*x₁ + 21*ν*x₁ - 2*ν*x₁^2 + 2*ν*y₁^2 - 41*ν*x₂*x₁ + 41*ν*y₁*y₂
 -60*y₂ + 41*y₁*x₂ + 41*y₂*x₁ + 160*y₂*x₂ + 60*ν*x₂ - 80*ν*x₂^2 + 80*ν*y₂^2 - 41*ν*x₂*x₁ + 41*ν*y₁*y₂
 System of length 12
 12 variables: x₁, x₂, y₁, y₂, a₁, a₂, b₁, b₂, λᴿ, λᴵ, s, t

 19 - 21*a₁ - 60*a₂ + 41*a₁*a₂ - 41*b₂*b₁ + a₁^2 + 40*a₂^2 

In [6]:
# the output solutions have form (a, b, t), only the real solutions are important
sols1 = real_solutions(solve(extended_systems[1]))
sols2 = real_solutions(solve(extended_systems[2]))
display(map(sol -> round.([sol[5:8]; sol[12]]; digits=4), [sols1; sols2]))

Tracking 256 paths... 100%|█████████████████████████████| Time: 0:00:02
  # paths tracked:                  256
  # non-singular solutions (real):  256 (16)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         256 (16)


32-element Vector{Vector{Float64}}:
 [0.4901, 0.5808, -0.2808, 0.1624, 1.0421]
 [0.4901, 0.5808, -0.2808, 0.1624, 0.4167]
 [0.4901, 0.5808, 0.2808, -0.1624, 1.0421]
 [0.4901, 0.5808, 0.2808, -0.1624, 0.4167]
 [0.4901, 0.5808, -0.2808, 0.1624, 0.6483]
 [0.4901, 0.5808, -0.2808, 0.1624, 688.0156]
 [0.4901, 0.5808, 0.2808, -0.1624, 0.6483]
 [0.4901, 0.5808, 0.2808, -0.1624, 688.0156]
 [9.9971, 0.2528, 0.0, 0.0, 30837.2464]
 [0.5482, 0.31, -0.0, -0.0, 2.4615]
 [0.5482, 0.31, 0.0, 0.0, 2.4615]
 [9.9971, 0.2528, -0.0, 0.0, 0.0095]
 [9.9971, 0.2528, 0.0, -0.0, 0.0085]
 ⋮
 [0.4901, 0.5808, -0.2808, 0.1624, 353.5263]
 [0.4901, 0.5808, -0.2808, 0.1624, 353.5263]
 [9.9971, 0.2528, 0.0, -0.0, 15588.2686]
 [0.4901, 0.5808, -0.2808, 0.1624, 0.5708]
 [0.4901, 0.5808, 0.2808, -0.1624, 0.5708]
 [0.5482, 0.31, -0.0, 0.0, 2.1184]
 [0.5482, 0.31, 0.0, 0.0, 4.9931]
 [0.5482, 0.31, 0.0, 0.0, 4.9931]
 [0.5482, 0.31, -0.0, 0.0, 2.1184]
 [0.4901, 0.5808, 0.2808, -0.1624, 0.5708]
 [0.4901, 0.5808, 0.2808, -0.16

## non-minimal points have $t \in (0, 1)$

In [7]:
certs1 = certificates(certify(extended_systems[1], sols1))
certs2 = certificates(certify(extended_systems[2], sols2))
intervals = [certified_solution_interval.(certs1); certified_solution_interval.(certs2)]
filtered = filter(I -> 0 < real(I[12]) < 1, intervals)
filtered = map(I -> midpoint.(real.(I)), filtered)
display(map(I -> [I[5:8]; I[12]], filtered))

10-element Vector{Vector{Arf}}:
 [0.49014901612649492, 0.58080333252729643, -0.28079160650357421, 0.16235478163384354, 0.41668698409890453]
 [0.49014901612649492, 0.58080333252729643, 0.28079160650357421, -0.16235478163384354, 0.41668698409890453]
 [0.49014901612649492, 0.58080333252729643, -0.28079160650357421, 0.16235478163384354, 0.64831209216753949]
 [0.49014901612649492, 0.58080333252729643, 0.28079160650357421, -0.16235478163384354, 0.64831209216753949]
 [9.9971105198210228, 0.25277497316471748, 0.0, 0.0, 0.0095053500541154093]
 [9.9971105198210228, 0.25277497316471748, 0.0, 0.0, 0.0084981950306754674]
 [0.49014901612649492, 0.58080333252729643, -0.28079160650357421, 0.16235478163384354, 0.57081777457152494]
 [0.49014901612649492, 0.58080333252729643, -0.28079160650357421, 0.16235478163384354, 0.57081777457152494]
 [0.49014901612649492, 0.58080333252729643, 0.28079160650357421, -0.16235478163384354, 0.57081777457152494]
 [0.49014901612649492, 0.58080333252729643, 0.28079160650357